# Групповой проект 2
[Соревнование на Kaggle](https://www.kaggle.com/c/ieee-fraud-detection/overview)
In this competition you are predicting the probability that an online transaction is fraudulent, as denoted by the binary target isFraud.

The data is broken into two files identity and transaction, which are joined by TransactionID. Not all transactions have corresponding identity information.

Categorical Features - Transaction
ProductCD
card1 - card6
addr1, addr2
P_emaildomain
R_emaildomain
M1 - M9
Categorical Features - Identity
DeviceType
DeviceInfo
id_12 - id_38
The TransactionDT feature is a timedelta from a given reference datetime (not an actual timestamp).


## Все импорты ноутбука здесь

In [9]:
import catboost as cb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import zipfile
import random
import os
import gc
from functools import partial
import requests
import io
import base64


## Загрузка данных

Files
- train_{transaction, identity}.csv - the training set
- test_{transaction, identity}.csv - the test set (you must predict the isFraud value for these observations)
- sample_submission.csv - a sample submission file in the correct format

> Загружаем данные из zip-архива, так как в распакованном виде они больше гигабайта

In [10]:
# Генерация ссылки на загрузку по share link onedrive
def create_onedrive_directdownload (onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl

In [11]:
%%time
def csv_from_zip(csv_file, zip_file='https://1drv.ms/u/s!AlUXsux_jwl0iYp1wTtAM_r5FyCpDQ', **kwargs):
   direct_download_url = create_onedrive_directdownload(zip_file)
   r = requests.get(direct_download_url) # url from my OneDrive
   with zipfile.ZipFile(io.BytesIO(r.content)) as z:
      with z.open(csv_file) as f:
         return pd.read_csv(f, **kwargs)

train_transaction = csv_from_zip('train_transaction.csv', delimiter=',')
train_identity = csv_from_zip('train_identity.csv', delimiter=',')

test_transaction = csv_from_zip('test_transaction.csv', delimiter=',')
test_identity = csv_from_zip('test_identity.csv', delimiter=',')

CPU times: user 1min 2s, sys: 10.1 s, total: 1min 12s
Wall time: 11min 42s


In [4]:
print(
    f'''
    Размеры выборок, (объекты, признаки).
    - Обучающие: transaction {train_transaction.shape}, identity {train_identity.shape}
    - Тестовые: transaction {test_transaction.shape}, identity {test_identity.shape}
    ''')


    Размеры выборок, (объекты, признаки).
    - Обучающие: transaction (590540, 394), identity (144233, 41)
    - Тестовые: transaction (506691, 393), identity (141907, 41)
    


## Разбиваем train_transaction на X и y. Где y - это целевая переменная

In [5]:
X = train_transaction.drop(["isFraud"],axis=1)
y = train_transaction["isFraud"]
del train_transaction
gc.collect

<function gc.collect(generation=2)>

## Полезные функции по обработке данных. Чтобы потом можно было их использовать в пайплайнах

In [6]:
# Установка seed для всех случайных счетчиков
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

SEED = 0xBEEF
seed_everything(SEED)

In [7]:
# Вспомогательная функция, которая позволяет объединять несколько шагов 
# по трансформации матриц (или других объектов) в один пайплайн
def make_transformer(*transformers):
    def transformer(*args):
        for t in transformers:
            try:
                args = t(args)
            except:
                args = t(*args)
        return args
    return transformer

In [24]:
# Убираем столбец(ы) с указанным именем
def drop_column(column):
    def drop(data):
        data.drop(column, axis=1, inplace=True)
        return data
    return drop

# Добавляем identity
def merge_identity(identity):
    def merge(data):
        return pd.merge(data, identity, how = 'left', on = 'TransactionID', validate = "many_to_one")
    return merge
    
# Cтолбцы, где слишком много пропущенных значений (выше определенного процента)
def nan_columns(data, nan_percent=0.8):
    nans = data.isna().sum()
    return nans[nans > nan_percent*len(data)].index

# Обработка категориальных признаков
# Конвертация float в string
# Замена пропущеных значений на строковые - 'nan'
def perform_cat_features(cat_features):
    def perform_cat(data):
        # Приведем все float категориальные фичи к строкам
        dtypes = data[cat_features].dtypes
        idx = dtypes == 'float64'
        cat_floats = dtypes[idx].index
        data[cat_floats] = data[cat_floats].astype(str)
        # Конвертируем все пропущенные значения в строки (для категориальных фич)
        data[cat_features] = data[cat_features].fillna('nan')
        return data
    return perform_cat

# В тестовой таблице identity ошибка в названии столбцов. Вместо id_ - id-
# Заменим в названии столбцов 'id'- на 'id_'
def fix_id_columns(data):
    data.columns = data.columns.str.replace('id-', 'id_')
    return data

# Кодируем категориальные признаки (простой способ Ordinal или Label)
def ordinal_encoding(columns):
    def encoding(data):
        oenc = preprocessing.OrdinalEncoder()
        data[columns] = oenc.fit_transform(data[columns])
        return data
    return encoding

## Базовая подготовка данных для catboost

### Добавляем identity 
1. Добавим к train (X) данные из identity
1. Удалим бесполезный столбец `TransactionID`

In [9]:
%%time
base_transform = make_transformer(
    merge_identity(train_identity),
    drop_column('TransactionID')
)
X = base_transform(X)
del train_identity
gc.collect;

CPU times: user 2.22 s, sys: 3.58 s, total: 5.81 s
Wall time: 6.98 s


<function gc.collect(generation=2)>

### Удаление столбцов, где слишком много пропущенных значений

In [10]:
bad_columns = nan_columns(X, nan_percent=0.8)
X.drop(bad_columns, axis=1, inplace=True)
print(f'Удалено {len(bad_columns)} столбцов, где было более 80% пропущеных значений')

Удалено 74 столбцов, где было более 80% пропущеных значений


### Категориальные и числовые признаки
[Обсуждение на Kaggle](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203#latest-607486)

#### Выделение категориальных и числовых признаков

In [11]:
cat_features = ['ProductCD'] + \
           ["card"+f"{i+1}" for i in range(6)] + \
           ["addr"+f"{i+1}" for i in range(2)] + \
           ["P_emaildomain", "R_emaildomain"] + \
           ["M"+f"{i+1}" for i in range(9)] + \
           ["DeviceType", "DeviceInfo"] + \
           ["id_"+f"{i}" for i in range(12, 39)]

# Вычтем удаленные ранее столбцы

cat_features = list(set(cat_features)- set(bad_columns))

In [12]:
num_features = list(set(X.columns)- set(cat_features))

In [13]:
print(f'Всего признаков = {X.shape[1]}, цифровых признаков = {len(num_features)}, категориальных = {len(cat_features)}')

Всего признаков = 358, цифровых признаков = 322, категориальных = 36


#### Подготовка категориальных признаков
cat_features must be integer or string, real number values and NaN values should be converted to string

In [14]:
# Приведем все float категориальные фичи к строкам
dtypes = X[cat_features].dtypes
idx = dtypes == 'float64'
cat_floats = dtypes[idx].index

In [15]:
X[cat_floats] = X[cat_floats].astype(str)

In [16]:
# Конвертируем все пропущенные значения в строки (для категориальных фич)
X[cat_features] = X[cat_features].fillna('nan')

## Базовый catboost

### Проверка с разбиением обучающей выборки

In [16]:
# Quick test with AUC

X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify = y)

cboost_params = {
    'loss_function': 'Logloss',
    'custom_loss':['AUC'],
    'logging_level':'Silent',
    # 'task_type' : 'GPU',
    # 'early_stopping_rounds' : 100
}

simple_model = cb.CatBoostClassifier(**cboost_params)

simple_model.fit(
    X_tr, y_tr,
    cat_features=cat_features,
    eval_set=(X_val, y_val),plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### Обучение catboost на всем train

In [22]:
# Обучение на всем train
train_dataset = cb.Pool(X, y, cat_features=cat_features)

cboost_params = {
    'loss_function': 'Logloss',
    'custom_loss':['AUC'],
}

model = cb.CatBoostClassifier(**cboost_params)
model.fit(train_dataset);

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.157129
0:	learn: 0.4638096	total: 3.65s	remaining: 1h 44s
1:	learn: 0.3236508	total: 7.45s	remaining: 1h 1m 57s
2:	learn: 0.2430940	total: 10.8s	remaining: 1h 2s
3:	learn: 0.1964945	total: 14s	remaining: 58m 4s
4:	learn: 0.1680217	total: 17.8s	remaining: 59m 8s
5:	learn: 0.1468356	total: 21.9s	remaining: 1h 23s
6:	learn: 0.1351064	total: 25.7s	remaining: 1h 42s
7:	learn: 0.1270535	total: 28.5s	remaining: 58m 57s
8:	learn: 0.1187904	total: 32.8s	remaining: 1h 6s
9:	learn: 0.1131566	total: 36s	remaining: 59m 21s
10:	learn: 0.1073625	total: 40.2s	remaining: 1h 14s
11:	learn: 0.1030373	total: 43.8s	remaining: 1h 9s
12:	learn: 0.0984045	total: 47.7s	remaining: 1h 21s
13:	learn: 0.0949658	total: 51.7s	remaining: 1h 44s
14:	learn: 0.0935842	total: 55.3s	remaining: 1h 31s
15:	learn: 0.0923722	total: 59.4s	remaining: 1h 52s
16:	learn: 0.0913112	total: 1m 3s	remaining: 1h 55s
17:	learn: 0.0874885	total: 1m 7s	remaining: 1h 1m 27s
18:	learn: 0.0862269	total: 1m 11s	remainin

### Сохранение модели в файл и загрузка из файла

In [ ]:
# Сохранение модели в файл
model.save_model('catboost 5.08.22')

In [20]:
# Загрузка модели из файла
from_file = cb.CatBoostClassifier()
from_file.load_model('catboost 5.08.22')

### Подготовка тестовой выборки

Мы должны сделать с тестовой выборкой все те преобразования, какие делали до этого с тренировочной

In [27]:
test_transformer = make_transformer(
    merge_identity(test_identity), # делаем добавление тестовой части identity
    drop_column('TransactionID'),
    fix_id_columns, # Исправляем ошибку в названии столбцов на test identity   
    drop_column(bad_columns), # Удаляем столбцы, где много пропущенных
    perform_cat_features(cat_features)
)

test = test_transformer(test_transaction)

In [18]:
test.shape, X.shape

((506691, 358), (590540, 358))

### Подготовка ответа для Kaggle

In [25]:
submission = csv_from_zip('sample_submission.csv', delimiter=',')
submission['isFraud'] = from_file.predict_proba(test)[:,1] # you must predict a probability for the isFraud variable
submission.to_csv('simple_catboost.csv', index=False)

## RandomForestClassifier
Посмотрим какой результат показывает случайный лес. И эта модель в любом случае может пригодиться нам для блендинга

### Подготовка данных
Случайный лес требует заполнения пропущенных значений и кодирования категориальных признаков

#### Заполнение пропущеных числовых признаков

Один из способов заполнения пропущеных - заполнить их каким-то числом, которое явно выбивается из распределения признака. Например, мы хотим заполнить числом -10 000. Чтобы понять подходит это число или нет - посмотрим на минимальные значения признаков (По тематике датасета отрицательных значений не должно быть или их должно быть мало)

In [25]:
min_X = X[num_features].min()
min_X.value_counts()

 0.000        308
 1.000          5
-100.000        2
-122.000        1
 0.251          1
-83.000         1
-72.000         1
 90.000         1
-53.000         1
 86400.000      1
dtype: int64

In [26]:
min_test = test[num_features].min()
min_test.value_counts()

 0.000000e+00    304
 1.000000e+00     11
-1.000000e+02      2
 1.800000e-02      1
 2.000000e+00      1
-8.100000e+01      1
 9.000000e+01      1
 1.840322e+07      1
dtype: int64

---
Вывод: мы можем заменить пропущенные числовые значения на -10 000

In [21]:
X = X.fillna(-10000)

In [1]:
test = test.fillna(-10000)

NameError: name 'test' is not defined

#### Кодируем категориальные переменные

In [2]:
X[cat_features]

NameError: name 'X' is not defined